In [71]:
# Install neccesary packages 

import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [72]:
# Read file 
data = pd.read_csv("Top_250_Movie.csv")

In [73]:
# Creating a column Views to update the given value 
data["views"] = ""
data.columns

Index(['ID', 'Title', 'Year', 'Rating ', 'Rating_count', 'views'], dtype='object')

In [74]:
# Update the given values to the new columns \

data.loc[data['Title'] == 'Forest Gump', 'views'] = 10000000
data.loc[data['Title'] == 'The Usual Suspects', 'views'] = 7500000
data.loc[data['Title'] == 'Rear Window', 'views'] = 6000000
data.loc[data['Title'] == 'North by Northwest', 'views'] = 4000000
data.loc[data['Title'] == 'The Secret in Their Eyes', 'views'] = 3000000
data.loc[data['Title'] == 'Spotlight', 'views'] = 1000000


In [75]:
# Preprocessing the data 

df = data.replace(',', '', regex = True) # Removing the commas to make str object to int
nan_value = float("NaN")
df.replace("", nan_value, inplace=True) # Replacing Nan values to empty cells 

In [76]:
# Create a seperate Dataframe to store values which has Nan
non_nan_df = df[df.isna().any(axis=1)] # Nan dataframe to predict

# Excluding the Nan values 
new = df.dropna() 

In [77]:
y = new['views']  # Dependant Variable  
x = new.drop(['ID', 'Title', 'Year', 'views'], axis=1) # Independant variables


In [78]:
# changing type to numeric 
x = x.apply(pd.to_numeric, errors='ignore')

In [79]:
# applying linear regression 
reg = LinearRegression().fit(x, y)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', n_estimators=1)
xg_reg.fit(x,y)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)

[10:18:45] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


RandomForestRegressor(max_depth=2, random_state=0)

In [80]:
non_nan_df['ID']

0      tt0111161
1      tt0068646
2      tt0071562
3      tt0468569
4      tt0050083
         ...    
245    tt0070047
246    tt0111495
247    tt0094625
248    tt1954470
249    tt2338151
Name: ID, Length: 245, dtype: object

In [81]:
Id_list = non_nan_df['ID'].to_list()

In [63]:
# df['views'].fillna(0, inplace=True)

In [82]:
# Storing id of movies to estimate it views 


df_pred = pd.DataFrame(columns = ['ID','Title','Year', 'Rating', 'Rating_count', 'views', 'XG_Boost_views','Linear_reg_views','Random_Forest_views'])
for i in Id_list:
    test1 = non_nan_df[(non_nan_df['ID'] == i)]
    test = test1[['Rating ', 'Rating_count']].apply(pd.to_numeric, errors='ignore')
    test1['XG_Boost_views'] = int(xg_reg.predict(test))
    test1['Linear_reg_views'] = int(abs(reg.predict(test)))
    test1['Random_Forest_views'] = int(regr.predict(test))

    df_pred.loc[i] = test1.values[0]


/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_97342/1611634196.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['XG_Boost_views'] = int(xg_reg.predict(test))
/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_97342/1611634196.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['Linear_reg_views'] = int(abs(reg.predict(test)))
/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_97342/1611634196.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [83]:
# Storing the final estimated value in pred.csv
df_pred.to_csv("Pred.csv") 

In [85]:
df_pred.tail

<bound method NDFrame.tail of                   ID                     Title  Year  Rating Rating_count  \
tt0111161  tt0111161  The Shawshank Redemption  1994     9.2      2089382   
tt0068646  tt0068646             The Godfather  1972     9.2      1433884   
tt0071562  tt0071562    The Godfather: Part II  1974     9.0       995997   
tt0468569  tt0468569           The Dark Knight  2008     9.0      2055819   
tt0050083  tt0050083              12 Angry Men  1957     8.9       591688   
...              ...                       ...   ...     ...          ...   
tt0070047  tt0070047              The Exorcist  1973     8.0       332258   
tt0111495  tt0111495         Three Colors: Red  1994     8.0        79786   
tt0094625  tt0094625                     Akira  1988     8.0       138660   
tt1954470  tt1954470        Gangs of Wasseypur  2012     8.0        68718   
tt2338151  tt2338151                        PK  2014     8.0       137003   

           views XG_Boost_views Linear_reg_vi